In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LdaModel
from gensim import corpora
import pandas as pd
import numpy as np
import gzip
import re
import string
from tqdm import tqdm
from time import time
# from hunspell import HunSpell
from multiprocessing import pool
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [61]:
df = getDF('data/reviews/reviews_Musical_Instruments.json.gz')

ans = df.reviewText + ' ' + df.summary

n = 10000
re_punctuation = re.compile('['+string.punctuation+']')
tokenizer = RegexpTokenizer('\w+')
stop = stopwords.words('english')
preprocessed_comments = []
for comment in tqdm(np.random.choice(ans, n)):
    comment = re_punctuation.sub(' ', comment)
    comment = tokenizer.tokenize(comment)
    comment = [x for x in comment if not any(c.isdigit() for c in x)]
    ########### HERE You have to insert HunSpell
    comment = [word for word in comment if word not in stop]
    preprocessed_comments.append(comment)
    
    
wordFrequency = Counter()
for comment in preprocessed_comments:
    wordFrequency.update(comment)                                  # Count overall word frequency
print('Unique Words In Comments: {}'.format(len(wordFrequency)))

minimumWordOccurrences = 5
# Remove rare words
texts = [[word for word in comment if wordFrequency[word] > minimumWordOccurrences] for comment in preprocessed_comments]

dictionary = corpora.Dictionary(texts)                             # Create word dictionary
vocabulary = [dictionary[i] for i in dictionary.keys()]
print('Documents/Comments: {}'.format(len(texts)))

corpus = [dictionary.doc2bow(doc) for doc in preprocessed_comments]                # Create corpus


100%|██████████| 10000/10000 [00:02<00:00, 3977.20it/s]

Unique Words In Comments: 34550
Documents/Comments: 10000


In [62]:
numberTopics = 20   #Number of topics

model_gensim = LdaModel(num_topics=numberTopics,
                        id2word=dictionary,
                        iterations=10,
                        passes=1,
                        chunksize=50,
                        alpha='auto',
                        eta='auto',
                        update_every=1)


perp_gensim = []
times_gensim = []
i=0
max_it = 5
min_prep = np.inf
start = time()

for _ in tqdm(range(100)):
    model_gensim.update(corpus)
    tmp = np.exp(-1 * model_gensim.log_perplexity(corpus))
    perp_gensim.append(tmp)
    times_gensim.append(time() - start)
    if(tmp<min_prep):
        min_prep = tmp;
    else:
        i = i + 1;
        if (i==max_it):
            break                # if prep increase for max_it number it will break the update procedure 
for i, topic in enumerate(model_gensim.get_topics().argsort(axis=1)[:, -10:][:, ::-1], 1):
    print('Topic {}: {}'.format(i, ' '.join([vocabulary[id] for id in topic])))


100%|██████████| 10/10 [01:43<00:00, 10.67s/it]

Topic 1: perfect hard found always item record others received original collection
Topic 2: Not full hear speaker fun volume performance Love speakers wood
Topic 3: works mic recording cable unit software USB worked perfectly plug
Topic 4: pedal violin clean beginner ones reason rest learn board video
Topic 5: The two work many one There first power less also
Topic 6: sound like really little better much good It sounds even
Topic 7: high low guitars top side solid Fender bridge All finish
Topic 8: keyboard drum Excellent days value change nothing kit next seem
Topic 9: They small These Amazon We tried big system read simple
Topic 10: set live band A built studio room mics times level
Topic 11: great It This price well quality good Great product nice
Topic 12: I one would time get bought case used could got
Topic 13: music piano CD album songs song recording heard track enjoy
Topic 14: guitar amp playing instrument play tone string end neck size
Topic 15: strings love He beautiful playe